In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
EMBEDDING_FILE = 'glove.840B.300d.txt'
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(test.shape)

(867, 6)


In [3]:
train["posts"]
test["posts"]
X_train = train["posts"]
y_train = train[["is_E", "is_S" , "is_T","is_J"]].values

X_test = test["posts"]

In [4]:
max_features=100000
maxlen=150
embed_size=300

In [5]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [6]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [7]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [9]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
preds = Dense(4, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [12]:
batch_size = 128
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [13]:
# filepath="best.hdf5"
filepath='best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

In [14]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)

Train on 7027 samples, validate on 781 samples
Epoch 1/2
7027/7027 [==============================] - 81s 12ms/step - loss: 0.5734 - accuracy: 0.7055 - val_loss: 0.5605 - val_accuracy: 0.7183

 ROC-AUC - epoch: 1 - score: 0.647603
Epoch 2/2


/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


7027/7027 [==============================] - 77s 11ms/step - loss: 0.5341 - accuracy: 0.7398 - val_loss: 0.5420 - val_accuracy: 0.7314

 ROC-AUC - epoch: 2 - score: 0.679395


/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [15]:
y_pred = model.predict(x_test,batch_size=1024,verbose=1)

867/867 [==============================] - 2s 3ms/step


In [16]:
print(x_test.shape)
print(y_pred.shape)

(867, 150)
(867, 4)


In [17]:
print(y_pred)

[[0.16694617 0.08990511 0.91886246 0.6233973 ]
 [0.14080909 0.0526481  0.20573759 0.16814572]
 [0.12866268 0.07045683 0.35209233 0.7381232 ]
 ...
 [0.2136862  0.14939398 0.49870065 0.49574396]
 [0.13830814 0.10618758 0.6948062  0.7489365 ]
 [0.66880375 0.12948129 0.8154876  0.22770321]]


In [18]:
submission = pd.read_csv('test.csv')
print(submission.shape)
submission[["is_E", "is_S" , "is_T","is_J"]] = y_pred

(867, 6)


In [19]:
submission.to_csv('submission.csv', index=False)